In [1]:
%load_ext autoreload
%autoreload 2

In [10]:
from glob import glob 
from fastcore.xtras import load_pickle

import pandas as pd

In [7]:
def get_prevalence(smiles, fragment):
    return sum([fragment in x for x in smiles]) / len(smiles)


In [3]:
all_res = glob('out/*/*.pkl')

In [4]:
all_res

['out/20230206_115656/summary.pkl', 'out/20230206_090056/summary.pkl']

In [15]:
temp_res

{'completions': {'choices': [' CC1=C(C(C)=NN1)/N=N/C2=CC=C(C(F)(F)F)C=C2',
   ' CC1=C(C(C)=NN1)/N=N/C2=CC=C(C(F)(F)F)C=C2',
   ' CC1=C(C(C)=NN1)/N=N/C2=CC=C(C(F)(F)F)C=C2',
   ' CC1=C(C(C)=NN1)/N=N/C2=CC=C(C(F)(F)F)C=C2',
   ' CC1=C(C(C)=NN1)/N=N/C2=CC=C(C(F)(F)F)C=C2',
   ' CC1=C(C(C)=NN1)/N=N/C2=CC=C(C(F)(F)F)C=C2',
   ' CC1=C(C(C)=NN1)/N=N/C2=CC=C(C(F)(F)F)C=C2',
   ' CC1=C(C(C)=NN1)/N=N/C2=CC=C(C(F)(F)F)C=C2',
   ' CC1=C(C(C)=NN1)/N=N/C2=CC=C(C(F)(F)F)C=C2',
   ' CC1=C(C(C)=NN1)/N=N/C2=CC=C(C(F)(F)F)C=C2',
   ' CC1=C(C(C)=NN1)/N=N/C2=CC=C(C(F)(F)F)C=C2',
   ' CC1=C(C(C)=NN1)/N=N/C2=CC=C(C(F)(F)F)C=C2',
   ' CC1=C(C(C)=NN1)/N=N/C2=CC=C(C(F)(F)F)C=C2',
   ' CC1=C(C(C)=NN1)/N=N/C2=CC=C(C(F)(F)F)C=C2',
   ' CC1=C(C(C)=NN1)/N=N/C2=CC=C(C(F)(F)F)C=C2',
   ' CC1=C(C(C)=NN1)/N=N/C2=CC=C(C(F)(F)F)C=C2',
   ' CC1=C(C(C)=NN1)/N=N/C2=CC=C(C(F)(F)F)C=C2',
   ' CC1=C(C(C)=NN1)/N=N/C2=CC=C(C(F)(F)F)C=C2',
   ' CC1=C(C(C)=NN1)/N=N/C2=CC=C(C(F)(F)F)C=C2',
   ' CC1=C(C(C)=NN1)/N=N/C2=CC=C(C(F)(F)F)C

In [16]:
compiled = []

for res in all_res:
    summary = load_pickle(res)
    for temp_res in summary['res_at_temp']:
        compiled.append({
            'train_size': summary['train_size'],
            'noise_level': summary['noise_level'],
            'num_samples': summary['num_samples'],
            'temperature': temp_res['temperature'],
            'valid_smiles': temp_res['valid_smiles'],
            'novel_smiles': temp_res['novel_smiles'],
            'valid_indices': temp_res['valid_indices'],
            'novel_indices': temp_res['novel_indices'],
            'frac_group': temp_res['fragment_fraction'],
            'constrain_satisfaction': temp_res['constrain_satisfaction'],
            'constrain_satisfaction_novel': temp_res['constrain_satisfaction_novel'],
            'prevalence': get_prevalence(temp_res['train_smiles'], summary['group']),
        })

In [17]:
compiled = pd.DataFrame(compiled)

In [20]:
compiled.groupby(['temperature']).mean()

/var/folders/m9/_txh68y946s4pxy1x2wnd3lh0000gn/T/ipykernel_53854/353237626.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  compiled.groupby(['temperature']).mean()


,train_size,noise_level,num_samples,frac_group,prevalence
temperature,,,,,
0.00,92.0,0.5,200.0,0.000000,0.01087
0.10,92.0,0.5,200.0,0.000000,0.01087
0.20,92.0,0.5,200.0,0.000000,0.01087
0.50,92.0,0.5,200.0,0.000000,0.01087
0.75,92.0,0.5,200.0,0.000000,0.01087
1.00,92.0,0.5,200.0,0.007692,0.01087
1.25,92.0,0.5,200.0,0.040833,0.01087
1.50,92.0,0.5,200.0,0.000000,0.01087
2.00,92.0,0.5,200.0,0.041667,0.01087
